In [1]:
import pandas as pd
import os
import sourmash, screed
import io
import urllib3

In [2]:
!wget --no-clobber https://osf.io/download/762mk/ -O sra.runinfo.csv.gz

File ‘sra.runinfo.csv.gz’ already there; not retrieving.


In [3]:
run_info = pd.read_csv('sra.runinfo.csv.gz')
print(f"Loaded {len(run_info)} records from sra.runinfo.csv.gz")
run_info.head()

Loaded 702013 records from sra.runinfo.csv.gz


,Unnamed: 0,Run,ScientificName
0,0,SRR18036904,bovine metagenome
1,1,SRR18036905,bovine metagenome
2,2,SRR18036906,bovine metagenome
3,3,SRR18036907,bovine metagenome
4,4,SRR18036908,bovine metagenome


In [4]:
# generate sourmash sketch
QUERY_SEQUENCE_FILE='sequences/shewanella.fa.gz'

total_bp = 0

sketch = sourmash.MinHash(0, 21, scaled=1000)
with screed.open(QUERY_SEQUENCE_FILE) as records:
    for record in records:
        sketch.add_sequence(record.sequence, force=True)
        total_bp += len(record.sequence)
        
print(f"generated {len(sketch)} hashes by sketching {total_bp:g} bp from '{QUERY_SEQUENCE_FILE}'")


generated 5307 hashes by sketching 5.31291e+06 bp from 'sequences/shewanella.fa.gz'


In [5]:
# serialize to SourmashSignature / gzipped JSON
ss = sourmash.SourmashSignature(sketch)

buf = io.BytesIO()
sourmash.save_signatures([ss], buf, compression=True)
print(f"serialized sourmash signature into {len(buf.getvalue())} bytes.")

serialized sourmash signature into 44044 bytes.


In [6]:
# query mastiff
http = urllib3.PoolManager()
r = http.request('POST',
                 'https://mastiff.sourmash.bio/search',
                 body = buf.getvalue(),
                 headers = { 'Content-Type': 'application/json'})

query_results_text = r.data.decode('utf-8')

In [7]:
# load results into a pandas data frame
results_wrap_fp = io.StringIO(query_results_text)
mastiff0_df = pd.read_csv(results_wrap_fp)

print(f"Loaded {len(mastiff0_df)} mastiff results into a dataframe!")

Loaded 5702 mastiff results into a dataframe!


In [8]:
mastiff0_df.head()

,SRA accession,containment
0,ERR1254284,1.000000
1,ERR1254310,1.000000
2,ERR1254285,0.999812
3,ERR1254311,0.999623
4,SRR606249,0.999246


In [9]:
mastiff0_df.tail()

,SRA accession,containment
5697,SRR11734631,0.009422
5698,SRR1217367,0.009422
5699,SRR10034926,0.009422
5700,SRR6448378,0.009422
5701,SRR10002862,0.009422


In [10]:
mastiff_df = mastiff0_df[mastiff0_df['containment'] >= 0.2]
print(f"Filtering results at a containment of >= 0.2; {len(mastiff_df)} of {len(mastiff0_df)} left.")

Filtering results at a containment of >= 0.2; 312 of 5702 left.


In [11]:
mastiff2_df = mastiff_df.set_index('SRA accession').join(run_info.set_index('Run')['ScientificName'])
mastiff2_df.head()

,containment,ScientificName
SRA accession,,
ERR1254284,1.000000,metagenome
ERR1254310,1.000000,metagenome
ERR1254285,0.999812,metagenome
ERR1254311,0.999623,metagenome
SRR606249,0.999246,synthetic metagenome


In [12]:
# how many have 'null' scientific name?
null_df = mastiff2_df[mastiff2_df['ScientificName'].isnull()]
print(len(null_df))

9


In [13]:
mastiff3_df = mastiff2_df[~mastiff2_df['ScientificName'].isnull()]
print(f"Of {len(mastiff2_df)} MAGsearch results, {len(mastiff3_df)} have non-null ScientificName")
mastiff3_df.head()

Of 312 MAGsearch results, 303 have non-null ScientificName


,containment,ScientificName
SRA accession,,
ERR1254284,1.000000,metagenome
ERR1254310,1.000000,metagenome
ERR1254285,0.999812,metagenome
ERR1254311,0.999623,metagenome
SRR606249,0.999246,synthetic metagenome


In [14]:
# what are the top ScientificNames of the matches?
mastiff3_df["ScientificName"].value_counts()[:20]

metagenome                  111
freshwater metagenome        32
wastewater metagenome        30
oyster metagenome            22
aquifer metagenome           17
marine metagenome            11
food metagenome              11
aquatic metagenome            9
biofilm metagenome            9
seawater metagenome           9
groundwater metagenome        6
synthetic metagenome          5
bioreactor metagenome         3
human skin metagenome         3
soil metagenome               2
money metagenome              2
fish metagenome               2
fish gut metagenome           2
oral metagenome               2
aquatic viral metagenome      2
Name: ScientificName, dtype: int64